In [2]:
import cv2
import dlib
from scipy.spatial import distance

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/Yawning_Detection/shape_predictor_68_face_landmarks.dat")

# Define mouth landmarks
MOUTH_START = 48
MOUTH_END = 68

# Define a function to calculate the aspect ratio of the mouth
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

# Open a video stream
video_capture = cv2.VideoCapture(0)  

In [10]:
predictor

In [ ]:
# Initialize variables
yawn_count = 0
yawn_status = False

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    # Iterate over the faces
    for face in faces:
        # Get the landmarks for the face
        landmarks = predictor(gray, face)

        # Extract the mouth landmarks
        mouth = []
        for i in range(MOUTH_START, MOUTH_END):
            x = landmarks.part(i).x
            y = landmarks.part(i).y
            mouth.append((x, y))

        # Calculate the aspect ratio of the mouth
        mar = mouth_aspect_ratio(mouth)

        # Check if the person is yawning
        if mar < 0.3:
            yawn_count += 1
            yawn_status = True
        else:
            yawn_status = False

        # Draw a rectangle around the face
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw a rectangle around the mouth
        cv2.rectangle(frame, (mouth[0][0], mouth[0][1]), (mouth[6][0], mouth[6][1]), (0, 0, 255), 2)

    # Display the yawn count and status
    font = cv2.FONT_HERSHEY_SIMPLEX
    if yawn_status:
        cv2.putText(frame, "Yawn Count: {}".format(yawn_count), (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, "Yawn Status: Yawning", (10, 70), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, "Yawn Count: {}".format(yawn_count), (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, "Yawn Status: Not Yawning", (10, 70), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Yawning Detection', frame)

    # Quit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break